# Keras による画像認識モデル開発と自動パラメータチューニング (Hyperdrive)

Keras を利用したモデル学習を行います。犬、猫の画像を区別する画像認識モデルを構築し、[Hyperdrive](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters) によるハイパーパラメータチューニングを実施します。

1. 事前準備
    - Python SDK のインポート
    - Azure ML `Workspace` への接続
    - `Experiment` の作成
    - `Dataset` の作成と登録
2. Keras による画像認識モデルの開発
3. 自動パラメータチューニング Hyperdrvie
    - 計算環境 `Machine Learning Compute` の準備
    - 自動パラメータチューニング Hyperdrive の事前設定
    - モデル学習と結果の確認

## 1. 事前準備

### Python SDK インポート
Azure Machine Learning python SDK をインポートします。

In [10]:
from azureml.core import Workspace, Dataset, Experiment

Azure Machine Learning Python SDK のバージョンを確認します。

In [11]:
print(azureml.core.VERSION)

1.0.76


### Azure Machine Learning Workspace への接続

In [14]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

azureml-ws	japaneast	azureml	japaneast


### 実験名の設定
Azure Machine Learing では 実験を管理する仕組みがあります。自動機械学習は自動的にその実験管理の仕組みでメトリックやログが残ります。

In [15]:
experiment = Experiment(workspace = workspace, name = "keras_catdog_hyperdrive")

### 学習データの準備

Azure Machine Learning service の計算環境 (Machine Learning Compute) で学習を回すために、Azure Machine Learning の Dataset のフォーマットでデータを定義します。

※ 利用するデータは Workshop 当日に講師に聞いてください。

In [18]:
dogcat = Dataset.get_by_name(workspace, name='dogcat')

### 計算環境 (Machine Learning Compute) の設定
Azure Machine Learning では機械学習のための計算環境 Machine Learning Compute が利用できます。

In [21]:
# 予め Azure ML studio にて gpu-cluster という名称の Machine Learning Compute を 作成しておく
from azureml.core.compute import ComputeTarget
compute_target = ComputeTarget(workspace,"gpu-cluster")

### モデル学習 Pythonスクリプト

In [23]:
import os
project_folder="./keras_hyperdrive"
os.makedirs(project_folder, exist_ok=True)

In [25]:
%%writefile {project_folder}/keras_dogcat.py

import numpy as np
import tensorflow as tf

from PIL import Image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import array_to_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Input, Dense
from tensorflow.python.keras.models import Model
import os
from azureml.core import Run
from azureml.core import Workspace, Dataset
from keras.utils import plot_model
import argparse

print("## START Script ##")


parser = argparse.ArgumentParser()
parser.add_argument('--batch-size', type=int, dest='batch_size', default=16, help='mini batch size for training')
parser.add_argument('--learning-rate', type=float, dest='learning_rate', default=0.01, help='learning rate')
parser.add_argument('--dropout', type=float, dest='dropout', default=0.01, help='dropout rate')
parser.add_argument('--epoch', type=int, dest='epoch', default=10, help='number of epoch')


args = parser.parse_args()



batch_size = args.batch_size
learing_rate = args.learning_rate
dropout = args.dropout
epoch = args.epoch

run = Run.get_context()
run.input_datasets['dogcat'].download(target_path='.', overwrite=False)


print(os.listdir())
print(os.listdir("train"))
print(os.listdir("test"))
print(os.listdir("train/cat"))
print(os.listdir("test/cat"))
print(os.listdir("train/dog"))
print(os.listdir("test/dog"))


from pip._internal.operations.freeze import freeze

class RunCallback(tf.keras.callbacks.Callback):
    def __init__(self, run):
        self.run = run
        
    def on_epoch_end(self, batch, logs={}):
        self.run.log(name="training_acc", value=float(logs.get('acc')))
        self.run.log(name="validation_acc", value=float(logs.get('val_acc')))
    

inputs = Input(shape=(150, 150, 3))
x = Conv2D(32, (3, 3))(inputs)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3))(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3))(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(dropout)(x)
prediction = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=prediction)
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])


callbacks = list()
callbacks.append(RunCallback(run))

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    './train',
    target_size=(150, 150), # resize
    batch_size=batch_size,
    class_mode="binary")

validation_generator = test_datagen.flow_from_directory(
     './test',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode="binary")

VERBOSE = 1

print("## START TRAINING ##")

model.fit_generator(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=epoch,
    validation_data=validation_generator,
    validation_steps=800 // batch_size,
    callbacks= callbacks)

run.log("Batch size",batch_size)
run.log("Num of Epoch", epoch)
run.log("Dropout Rate", dropout)

model.save_weights("./outputs/keras_simple.h5")
#plot_model(model, to_file='./outputs/model.png')


Overwriting ./keras_hyperdrive/keras_dogcat.py


## 2. Keras による画像認識モデルの開発

### TensorFlow Estimator 設定

Keras(TensorFlow backend) を用いたモデル学習の事前設定を行います。[TensorFlow Estimator](https://docs.microsoft.com/ja-JP/python/api/azureml-train-core/azureml.train.dnn.tensorflow) を用いて定義します。

In [29]:
from azureml.train.dnn import TensorFlow

script_params = {
    '--batch-size': 16,
    '--learning-rate': 0.00001,
    '--dropout': 0.03,
    '--epoch':20
}


estimator = TensorFlow(source_directory=project_folder,
                       script_params = script_params,
                       compute_target=compute_target,
                       entry_script='keras_dogcat.py',
                       framework_version = '1.13',
                       pip_packages=['keras','Pillow','azureml-dataprep[pandas,fuse]'],
                       inputs=[dogcat.as_named_input('dogcat')]
                      )

### モデル学習と結果確認
一番最初の実行は Docker Image を作成するため、20 〜 30分ほど時間がかかります。Dokcer Image を Build している様子は Azure Machine Learning studio から確認できます。

In [30]:
run = experiment.submit(estimator)

In [34]:
from azureml.widgets import RunDetails
RunDetails(run).show() 

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…

In [32]:
# # 詳細ログの出力
# run.get_details()

## 3. 自動パラメータチューニング Hyperdrvie

### Hyperdrive ハイパーパラメータチューニング
自動パラメータチューニングの事前設定を行います。[HyperDriveConfig](https://docs.microsoft.com/ja-jp/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py) を用いて定義します。

In [37]:
from azureml.train.dnn import TensorFlow

tf_hyperdrive_est = TensorFlow(source_directory=project_folder,
                       #script_params = script_params,
                       compute_target=compute_target,
                       entry_script='keras_dogcat.py',
                       framework_version = '1.13',
                       pip_packages=['keras','Pillow','azureml-dataprep[pandas,fuse]'],
                       inputs=[dogcat.as_named_input('dogcat')]
                      )

HyperDrive が探索パラメータの範囲を定義します。詳細は  [Azure Machine Learning でモデルのハイパーパラメーターを調整する](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-tune-hyperparameters) をご参照ください。

In [40]:
from azureml.train.hyperdrive import *

ps = RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 75,100),
        '--learning-rate': loguniform(-6, -1),
        '--dropout': loguniform(-6, -1),
        '--epoch' : choice(range(10,100))
    }
)

In [41]:
# 早期停止ポリシー (2イテレーション毎チェック、トップ10%未満の場合は停止)
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [42]:
htc = HyperDriveConfig(estimator=tf_hyperdrive_est, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name='validation_acc', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=4,  # トータル試行回数
                          max_concurrent_runs=4)  # 最大並列度

## Hyperdrive によるモデル学習と結果確認

In [43]:
htr = experiment.submit(config=htc)

In [48]:
RunDetails(htr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSE…

In [49]:
# # 詳細ログの出力
# htr.get_details()

## モデル登録
一番精度が良かったモデルを Azure Machine Learning に登録します。

In [50]:
# HyperDrive によるモデル学習完了後に実行すること
best_run = htr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)

Run(Experiment: keras_catdog_hyperdrive,
Id: keras_catdog_hyperdrive_1577438351727707_2,
Type: azureml.scriptrun,
Status: Completed)


In [51]:
model = best_run.register_model(model_name='tf-catdog-hyperdrive', model_path='outputs/')
print(model.name, model.id, model.version, sep = '\t')

tf-catdog-hyperdrive	tf-catdog-hyperdrive:1	1


In [52]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
keras_catdog_hyperdrive,keras_catdog_hyperdrive_1577438042_ea14e851,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation
